In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
print(dataset.corr())
print(dataset.describe())

print(dataset.mean())
print(dataset.std())
print(meanFeatures)
#dataset.head(100)
#It make sense there are not nulls

In [ ]:

shuffledDataSet = dataset.reindex(np.random.permutation(dataset.index)) #I mix the dataset to divide it after
#shuffledDataSet.head(100)
catColumns = ["sex", "smoker", "region"]
shuffledDataSet = pd.get_dummies(shuffledDataSet, columns = catColumns, drop_first=True)
def normalize(x):
  return (x - x.mean())/x.std()


def featuresAndLabels(dataset,label):
  features = dataset.copy()
  labels = features.pop(label)
  features = (features-features.mean()) / features.std()
  dict = { 'labels' : labels, 'features' : features}
  return dict


size = shuffledDataSet.size
forTesting= 0.2
label='expenses'

trainDataSet = shuffledDataSet.head(size-int(size*forTesting))
trainDataSet = featuresAndLabels(trainDataSet, 'expenses') 

testDataSet = shuffledDataSet.tail(int(size*forTesting))
testDataSet = featuresAndLabels(testDataSet, 'expenses') 

testDataSet['features']
#testDataSet['label']


In [ ]:
def train_model(model, df, my_epochs, 
                my_batch_size=1,):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=df['features'],
                      y=df['labels'],
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=0.1)



  # The list of epochs is stored separately from the 
  # rest of history.
  #epochs = history.epoch
  
  # Isolate the root mean squared error for each epoch.
  #hist = pd.DataFrame(history.history)
  #rmse = hist["root_mean_squared_error"]

  #return epochs, rmse, history.history   

In [ ]:
inputShape = trainDataSet['features'].shape[1]
print(inputShape)


my_learning_rate=0.5

model = keras.Sequential()
model.add( layers.Dense(units=1,input_shape = (inputShape,) )  )
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
              loss='mae',
              metrics=['mae','mse'])

In [ ]:
train_model(model,trainDataSet,1000,my_batch_size=4)

In [ ]:
test_dataset = testDataSet['features']
test_labels = testDataSet['labels']

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()
print(test_predictions)

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
